In [1]:
import matplotlib.pyplot as plt

import torchvision as tv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.optim as optim
import os
import time
import loadData8
import itertools
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
device = torch.device("cuda:0")

In [3]:
transforms = tv.transforms.Compose([
    # transforms.CenterCrop(64),
    # transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [4]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(20304, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net8 = Net8()
path = './model_30dB_8个0一组.pkl'
net8.load_state_dict(torch.load(path))
net8 = net8.to(device)

In [5]:
class Net0(nn.Module):
    def __init__(self):
        super(Net0, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(2384, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net0 = Net0()
path = './model_30dB_1个0一组.pkl'
net0.load_state_dict(torch.load(path))
net0 = net0.to(device)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(10064, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

net = Net()
path = './model_30dB_4个0一组.pkl'
net.load_state_dict(torch.load(path))
net = net.to(device)

In [7]:
predict1, predict2, predict3 = [],[],[]
originLabels = []

In [8]:
import deleteFolder
for line in open("files_test低延迟室外.txt"):   
    print(line)
    lines = line.split()
    locate, label = lines[0], lines[1]
    originData = np.load(locate)
    
    #测试8个0的情况
    with open("./split-no.txt" ,"w") as f8:
        f8.write(line)
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split-no.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)

    for data in test_loader:
        inputs, labels = data
        originLabel = int(labels)
        originLabels.append(originLabel)

        a = inputs[0:160]
#         print("---", a.shape)
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net8(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predict1.append(int(predicted[0]))
#         print(labels,"---", predicted)
    
    
    #测试输入1个0的情况
    splitData = np.split(originData,8,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/输入1个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
        
    with open("./split8.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/输入1个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
            
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split8.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net0(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
#     predict2.append(np.argmax(np.bincount(predictedLabels)))
    predict2.append([predictedLabel for predictedLabel in predictedLabels])
        
    #测试输入4个0的情况
    originData = np.load(locate)
    splitData = np.split(originData,2,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/输入4个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
    
    with open("./split2.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/输入4个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split2.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
    predict3.append([p for p in predictedLabels]) 
   
    
    print(originLabels[-1],  predict2[-1], predict3[-1], predict1[-1])

/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_115.npy 9



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


9 [50, 50, 32, 50, 36, 50, 50, 50] [36, 50] 32
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_116.npy 9



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


9 [50, 50, 45, 45, 50, 50, 50, 50] [30, 30] 32
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_117.npy 9

9 [50, 45, 50, 50, 50, 50, 50, 50] [30, 30] 32
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_118.npy 9

9 [50, 50, 50, 32, 50, 49, 32, 32] [36, 30] 32
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_119.npy 9

9 [50, 50, 50, 45, 32, 50, 32, 50] [45, 30] 32
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_120.npy 9

9 [50, 32, 32, 34, 50, 50, 50, 34] [45, 32] 36
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_145.npy 9

9 [50, 50, 50, 50, 32, 50, 50, 50] [45, 30] 36
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_146.npy 9

9 [50, 50, 50, 50, 50, 50, 50, 50] [32, 30] 41
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_147.npy 9

9 [50, 50, 50, 50, 50, 50, 50, 50] [50, 32] 41
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_148.npy 9

9 [50, 50, 50, 50, 13, 50, 50, 50] [32, 30] 32
/workspace/瑞利多径/八个0的情况/低延迟室外处理后的数据/Process_A_10_149.npy 9

9 [50, 50, 34, 50, 50, 50, 50, 50] [30, 30] 36

In [9]:
rightNum = 0
for i in range(len(originLabels)):
    predictAll = []
    predictAll.append(predict1[i])
    for a in predict2[i]:
        predictAll.append(a)
    for a in predict3[i]:
        predictAll.append(a)
#     print("-----",predictAll)
#     print(p.shape)
    result = np.argmax(np.bincount(predictAll))
#     print(result, originLabels[i])
    if originLabels[i] == result:
        rightNum += 1
    else:
        print(originLabels[i], predict1[i], predict2[i], predict3[i])
print(rightNum/ len(originLabels))

9 32 [50, 50, 32, 50, 36, 50, 50, 50] [36, 50]
9 32 [50, 50, 45, 45, 50, 50, 50, 50] [30, 30]
9 32 [50, 45, 50, 50, 50, 50, 50, 50] [30, 30]
9 32 [50, 50, 50, 32, 50, 49, 32, 32] [36, 30]
9 32 [50, 50, 50, 45, 32, 50, 32, 50] [45, 30]
9 36 [50, 32, 32, 34, 50, 50, 50, 34] [45, 32]
9 36 [50, 50, 50, 50, 32, 50, 50, 50] [45, 30]
9 41 [50, 50, 50, 50, 50, 50, 50, 50] [32, 30]
9 41 [50, 50, 50, 50, 50, 50, 50, 50] [50, 32]
9 32 [50, 50, 50, 50, 13, 50, 50, 50] [32, 30]
9 36 [50, 50, 34, 50, 50, 50, 50, 50] [30, 30]
9 32 [50, 32, 50, 32, 50, 50, 45, 50] [30, 32]
9 32 [50, 50, 50, 50, 50, 45, 50, 50] [32, 34]
9 41 [50, 34, 34, 50, 50, 50, 50, 32] [32, 45]
9 41 [50, 50, 50, 32, 32, 45, 36, 45] [36, 30]
9 36 [50, 32, 32, 50, 34, 50, 34, 36] [32, 30]
9 41 [34, 50, 34, 50, 50, 50, 49, 45] [30, 30]
9 36 [45, 32, 32, 32, 32, 32, 34, 50] [30, 30]
9 41 [50, 50, 50, 50, 50, 50, 50, 50] [30, 41]
9 32 [36, 45, 50, 50, 50, 50, 50, 45] [30, 30]
9 32 [45, 50, 50, 32, 50, 50, 50, 50] [30, 45]
9 32 [34, 50,